In [2]:
from scapy.layers.inet import *
from scapy.all import *

sr1(IP(dst="8.8.8.8")/ICMP()).show()

Begin emission:
Finished sending 1 packets.

Received 8 packets, got 1 answers, remaining 0 packets
###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 28
  id        = 0
  flags     = 
  frag      = 0
  ttl       = 121
  proto     = icmp
  chksum    = 0x84d6
  src       = 8.8.8.8
  dst       = 10.65.162.186
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0x0
     id        = 0x0
     seq       = 0x0
     unused    = ''
###[ Padding ]### 
        load      = '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



In [3]:
# Senden eines Pakets ohne auf eine Antwort zu warten
send(IP(dst="8.8.8.8")/ICMP())

# Senden eines Pakets und Empfangen der Antwort
ans, unans = sr(IP(dst="8.8.8.8")/ICMP())
ans.summary()


Sent 1 packets.
Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets
IP / ICMP 10.65.162.186 > 8.8.8.8 echo-request 0 ==> IP / ICMP 8.8.8.8 > 10.65.162.186 echo-reply 0 / Padding


In [ ]:
# Sniffen von Paketen auf dem Interface eth0
packets = sniff(iface="eth0", count=10)
packets.summary()
